In [37]:
import networkx as nx
import dgl
import numpy as np

In [38]:
import pandas as pd
rating_df = pd.read_csv('./data/raw_rating_data.csv', sep=',')
rating_df

,user,item,rating
0,1,104,2.0
1,1,105,5.0
2,1,106,5.0
3,1,107,5.0
4,1,108,5.0
...,...,...,...
356662,49103,16369,5.0
356663,49103,3425,5.0
356664,49103,66201,5.0
356665,49103,6889,5.0


In [39]:
link_df = pd.read_csv('./data/raw_link_data.csv', sep=',')
link_df

,user1,user2,weight
0,22605,5052,1.0
1,2824,2696,1.0
2,2824,18333,1.0
3,2824,10308,1.0
4,2824,1176,1.0
...,...,...,...
347935,615,1096,1.0
347936,615,145,1.0
347937,615,1499,1.0
347938,615,330,1.0


In [40]:
rating_df.user.value_counts().sort_index().index

Int64Index([    1,     2,     4,     5,     7,     8,     9,    11,    12,
               13,
            ...
            46912, 47123, 47189, 47671, 47868, 48047, 48213, 48612, 49013,
            49103],
           dtype='int64', length=12771)

In [41]:
link_df.user1.value_counts().sort_index().index

Int64Index([    1,     2,     4,     5,     7,     8,     9,    11,    12,
               13,
            ...
            46912, 47123, 47189, 47671, 47868, 48047, 48213, 48612, 49013,
            49103],
           dtype='int64', length=12532)

In [43]:
def process_link(df, name1='user1', name2='user2'):
    cpy = df.copy()
    cpy[name1], cpy[name2] = cpy[name2], cpy[name1]
    res = pd.concat([df, cpy], axis=0)
    return res.drop_duplicates().reset_index().drop(columns=['index']).copy()

In [44]:
bi_link_df = process_link(link_df)

In [45]:
# 保证rating和link里面的user一致
(~rating_df.user.value_counts().sort_index().index == bi_link_df.user1.value_counts().sort_index().index).sum()

0

In [46]:
item_N = len(rating_df.item.value_counts())
item_N

23143

In [47]:
user_N = len(rating_df.user.value_counts())
user_N

12771

In [48]:
u_array = np.arange(user_N)
u_array

array([    0,     1,     2, ..., 12768, 12769, 12770])

In [49]:
i_array = np.arange(user_N, user_N + item_N)
i_array

array([12771, 12772, 12773, ..., 35911, 35912, 35913])

In [50]:
# 生成字典
sorted_user_idx_lst = sorted(rating_df.user.value_counts().index)
sorted_user_idx_lst

[1,
 2,
 4,
 5,
 7,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 19,
 20,
 21,
 22,
 23,
 24,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 43,
 45,
 46,
 47,
 48,
 50,
 51,
 53,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 65,
 66,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 98,
 99,
 102,
 103,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 125,
 127,
 128,
 129,
 130,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 169,
 171,
 172,
 173,
 174,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 195,
 197,
 198,
 199,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 218,


In [51]:
user_raw_idx2new_idx = {}
user_new_idx2raw_idx = {}
for i in range(len(sorted_user_idx_lst)):
    raw_id = sorted_user_idx_lst[i]
    new_id = int(u_array[i])
    # print(type(raw_id), type(new_id))
    user_raw_idx2new_idx[raw_id] = new_id
    user_new_idx2raw_idx[new_id] = raw_id
    # print(user_raw_idx2new_idx)

In [52]:
user_raw_idx2new_idx

{1: 0,
 2: 1,
 4: 2,
 5: 3,
 7: 4,
 8: 5,
 9: 6,
 11: 7,
 12: 8,
 13: 9,
 14: 10,
 15: 11,
 16: 12,
 19: 13,
 20: 14,
 21: 15,
 22: 16,
 23: 17,
 24: 18,
 27: 19,
 28: 20,
 29: 21,
 30: 22,
 31: 23,
 32: 24,
 33: 25,
 34: 26,
 35: 27,
 36: 28,
 38: 29,
 39: 30,
 40: 31,
 41: 32,
 42: 33,
 43: 34,
 45: 35,
 46: 36,
 47: 37,
 48: 38,
 50: 39,
 51: 40,
 53: 41,
 56: 42,
 57: 43,
 58: 44,
 59: 45,
 60: 46,
 61: 47,
 62: 48,
 65: 49,
 66: 50,
 67: 51,
 68: 52,
 69: 53,
 71: 54,
 72: 55,
 73: 56,
 74: 57,
 76: 58,
 77: 59,
 78: 60,
 79: 61,
 80: 62,
 81: 63,
 82: 64,
 84: 65,
 85: 66,
 86: 67,
 87: 68,
 89: 69,
 90: 70,
 91: 71,
 92: 72,
 93: 73,
 94: 74,
 95: 75,
 96: 76,
 98: 77,
 99: 78,
 102: 79,
 103: 80,
 105: 81,
 106: 82,
 108: 83,
 109: 84,
 110: 85,
 111: 86,
 112: 87,
 114: 88,
 115: 89,
 116: 90,
 117: 91,
 118: 92,
 120: 93,
 121: 94,
 122: 95,
 123: 96,
 124: 97,
 125: 98,
 127: 99,
 128: 100,
 129: 101,
 130: 102,
 132: 103,
 133: 104,
 134: 105,
 135: 106,
 136: 107,
 137: 10

In [53]:
user_new_idx2raw_idx

{0: 1,
 1: 2,
 2: 4,
 3: 5,
 4: 7,
 5: 8,
 6: 9,
 7: 11,
 8: 12,
 9: 13,
 10: 14,
 11: 15,
 12: 16,
 13: 19,
 14: 20,
 15: 21,
 16: 22,
 17: 23,
 18: 24,
 19: 27,
 20: 28,
 21: 29,
 22: 30,
 23: 31,
 24: 32,
 25: 33,
 26: 34,
 27: 35,
 28: 36,
 29: 38,
 30: 39,
 31: 40,
 32: 41,
 33: 42,
 34: 43,
 35: 45,
 36: 46,
 37: 47,
 38: 48,
 39: 50,
 40: 51,
 41: 53,
 42: 56,
 43: 57,
 44: 58,
 45: 59,
 46: 60,
 47: 61,
 48: 62,
 49: 65,
 50: 66,
 51: 67,
 52: 68,
 53: 69,
 54: 71,
 55: 72,
 56: 73,
 57: 74,
 58: 76,
 59: 77,
 60: 78,
 61: 79,
 62: 80,
 63: 81,
 64: 82,
 65: 84,
 66: 85,
 67: 86,
 68: 87,
 69: 89,
 70: 90,
 71: 91,
 72: 92,
 73: 93,
 74: 94,
 75: 95,
 76: 96,
 77: 98,
 78: 99,
 79: 102,
 80: 103,
 81: 105,
 82: 106,
 83: 108,
 84: 109,
 85: 110,
 86: 111,
 87: 112,
 88: 114,
 89: 115,
 90: 116,
 91: 117,
 92: 118,
 93: 120,
 94: 121,
 95: 122,
 96: 123,
 97: 124,
 98: 125,
 99: 127,
 100: 128,
 101: 129,
 102: 130,
 103: 132,
 104: 133,
 105: 134,
 106: 135,
 107: 136,
 108: 13

In [54]:
import json
user_idx_dict = {
    'raw2new': user_raw_idx2new_idx,
    'new2raw': user_new_idx2raw_idx
}
with open('./data/user_idx_transfer.json', 'w') as f:
    json.dump(user_idx_dict, f, indent=2)

In [55]:
# 生成字典
sorted_item_idx_lst = sorted(rating_df.item.value_counts().index)
sorted_item_idx_lst

[1,
 18,
 36,
 40,
 44,
 47,
 61,
 67,
 69,
 74,
 77,
 78,
 79,
 80,
 87,
 89,
 90,
 100,
 104,
 105,
 106,
 107,
 108,
 109,
 112,
 113,
 115,
 116,
 117,
 122,
 125,
 128,
 129,
 132,
 133,
 134,
 137,
 138,
 139,
 140,
 143,
 144,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 159,
 160,
 162,
 164,
 165,
 166,
 173,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 184,
 187,
 188,
 192,
 195,
 201,
 202,
 203,
 212,
 213,
 214,
 215,
 216,
 217,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 231,
 232,
 233,
 237,
 242,
 243,
 252,
 255,
 268,
 269,
 272,
 283,
 295,
 296,
 309,
 322,
 332,
 333,
 334,
 335,
 336,
 341,
 355,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 367,
 368,
 369,
 370,
 371,
 372,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 382,
 384,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 400,
 401,
 405,
 413,
 419,
 423,
 429,
 430,
 434,
 437,
 454,
 466,
 467,
 468,
 471,
 476,
 479,
 480,
 486,
 487,
 488,
 489,
 490,
 492,
 493,
 495,
 498

In [56]:
item_raw_idx2new_idx = {}
item_new_idx2raw_idx = {}
for i in range(len(sorted_item_idx_lst)):
    raw_id = sorted_item_idx_lst[i]
    new_id = int(i_array[i])
    item_raw_idx2new_idx[raw_id] = new_id
    item_new_idx2raw_idx[new_id] = raw_id

In [57]:
item_raw_idx2new_idx

{1: 12771,
 18: 12772,
 36: 12773,
 40: 12774,
 44: 12775,
 47: 12776,
 61: 12777,
 67: 12778,
 69: 12779,
 74: 12780,
 77: 12781,
 78: 12782,
 79: 12783,
 80: 12784,
 87: 12785,
 89: 12786,
 90: 12787,
 100: 12788,
 104: 12789,
 105: 12790,
 106: 12791,
 107: 12792,
 108: 12793,
 109: 12794,
 112: 12795,
 113: 12796,
 115: 12797,
 116: 12798,
 117: 12799,
 122: 12800,
 125: 12801,
 128: 12802,
 129: 12803,
 132: 12804,
 133: 12805,
 134: 12806,
 137: 12807,
 138: 12808,
 139: 12809,
 140: 12810,
 143: 12811,
 144: 12812,
 149: 12813,
 150: 12814,
 151: 12815,
 152: 12816,
 153: 12817,
 154: 12818,
 155: 12819,
 156: 12820,
 157: 12821,
 159: 12822,
 160: 12823,
 162: 12824,
 164: 12825,
 165: 12826,
 166: 12827,
 173: 12828,
 176: 12829,
 177: 12830,
 178: 12831,
 179: 12832,
 180: 12833,
 181: 12834,
 182: 12835,
 184: 12836,
 187: 12837,
 188: 12838,
 192: 12839,
 195: 12840,
 201: 12841,
 202: 12842,
 203: 12843,
 212: 12844,
 213: 12845,
 214: 12846,
 215: 12847,
 216: 12848,
 217

In [58]:
item_new_idx2raw_idx

{12771: 1,
 12772: 18,
 12773: 36,
 12774: 40,
 12775: 44,
 12776: 47,
 12777: 61,
 12778: 67,
 12779: 69,
 12780: 74,
 12781: 77,
 12782: 78,
 12783: 79,
 12784: 80,
 12785: 87,
 12786: 89,
 12787: 90,
 12788: 100,
 12789: 104,
 12790: 105,
 12791: 106,
 12792: 107,
 12793: 108,
 12794: 109,
 12795: 112,
 12796: 113,
 12797: 115,
 12798: 116,
 12799: 117,
 12800: 122,
 12801: 125,
 12802: 128,
 12803: 129,
 12804: 132,
 12805: 133,
 12806: 134,
 12807: 137,
 12808: 138,
 12809: 139,
 12810: 140,
 12811: 143,
 12812: 144,
 12813: 149,
 12814: 150,
 12815: 151,
 12816: 152,
 12817: 153,
 12818: 154,
 12819: 155,
 12820: 156,
 12821: 157,
 12822: 159,
 12823: 160,
 12824: 162,
 12825: 164,
 12826: 165,
 12827: 166,
 12828: 173,
 12829: 176,
 12830: 177,
 12831: 178,
 12832: 179,
 12833: 180,
 12834: 181,
 12835: 182,
 12836: 184,
 12837: 187,
 12838: 188,
 12839: 192,
 12840: 195,
 12841: 201,
 12842: 202,
 12843: 203,
 12844: 212,
 12845: 213,
 12846: 214,
 12847: 215,
 12848: 216,
 128

In [59]:
item_idx_dict = {
    'raw2new': item_raw_idx2new_idx,
    'new2raw': item_new_idx2raw_idx
}
with open('./data/item_idx_transfer.json', 'w') as f:
    json.dump(item_idx_dict, f, indent=2)

# 将原始数据的idx替换

In [60]:
new_rating_df = rating_df.copy()

In [61]:
new_rating_df

,user,item,rating
0,1,104,2.0
1,1,105,5.0
2,1,106,5.0
3,1,107,5.0
4,1,108,5.0
...,...,...,...
356662,49103,16369,5.0
356663,49103,3425,5.0
356664,49103,66201,5.0
356665,49103,6889,5.0


In [62]:
new_rating_df['user'] = new_rating_df['user'].apply(lambda x: user_raw_idx2new_idx[x])
new_rating_df

,user,item,rating
0,0,104,2.0
1,0,105,5.0
2,0,106,5.0
3,0,107,5.0
4,0,108,5.0
...,...,...,...
356662,12770,16369,5.0
356663,12770,3425,5.0
356664,12770,66201,5.0
356665,12770,6889,5.0


In [63]:
new_rating_df['item'] = new_rating_df['item'].apply(lambda x: item_raw_idx2new_idx[x])
new_rating_df

,user,item,rating
0,0,12789,2.0
1,0,12790,5.0
2,0,12791,5.0
3,0,12792,5.0
4,0,12793,5.0
...,...,...,...
356662,12770,19494,5.0
356663,12770,14287,5.0
356664,12770,31360,5.0
356665,12770,15691,5.0


In [64]:
new_rating_df.to_csv('./data/rating_data.csv', index=False, header=True)

In [65]:
new_link_df = link_df.copy()
new_link_df

,user1,user2,weight
0,22605,5052,1.0
1,2824,2696,1.0
2,2824,18333,1.0
3,2824,10308,1.0
4,2824,1176,1.0
...,...,...,...
347935,615,1096,1.0
347936,615,145,1.0
347937,615,1499,1.0
347938,615,330,1.0


In [66]:
new_link_df['user1'] = new_link_df['user1'].apply(lambda x: user_raw_idx2new_idx[x])
new_link_df

,user1,user2,weight
0,11098,5052,1.0
1,2374,2696,1.0
2,2374,18333,1.0
3,2374,10308,1.0
4,2374,1176,1.0
...,...,...,...
347935,525,1096,1.0
347936,525,145,1.0
347937,525,1499,1.0
347938,525,330,1.0


In [67]:
new_link_df['user2'] = new_link_df['user2'].apply(lambda x: user_raw_idx2new_idx[x])
new_link_df

,user1,user2,weight
0,11098,3974,1.0
1,2374,2278,1.0
2,2374,9820,1.0
3,2374,6751,1.0
4,2374,1045,1.0
...,...,...,...
347935,525,970,1.0
347936,525,115,1.0
347937,525,1299,1.0
347938,525,283,1.0


In [68]:
new_link_df.to_csv('./data/link_data.csv', index=False, header=True)